### **This script generates descriptors for the classification task. If you want to use it for regression, just change it a little bit according to the comments in the appropriate code blocks**

In [2]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 12.5 MB/s eta 0:00:00


In [48]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams

from rdkit import Chem

from rdkit.Chem import Crippen, Descriptors, GraphDescriptors, Lipinski, QED, rdMolDescriptors, Fragments, FragmentMatcher
from rdkit.Chem.EState.EState_VSA import VSA_EState_
from tqdm import tqdm

In [49]:
import sklearn
from sklearn.feature_selection import SelectKBest, mutual_info_classif, mutual_info_regression

In [50]:
filename = 'classification_dataset_without_descriptors.csv' # write the name of your file here
df = pd.read_csv(filename)
df

,SMILES,Activity,Agglomeration,SMILES_uncharge,FORMAL_CHARGE_unch
0,BrC(Br)Br,1,False,BrC(Br)Br,0
1,C#CC(C)(O)CC,1,False,C#CC(C)(O)CC,0
2,C#CC(O)(/C=C/Cl)CC,1,False,C#CC(O)(/C=C/Cl)CC,0
3,C#CC(OC(N)=O)c1ccccc1,1,False,C#CC(OC(N)=O)C1=CC=CC=C1,0
4,C#CC1(OCC(O)CN2CCN(c3ccc(F)cc3)CC2)CCCCC1,1,False,C#CC1(OCC(O)CN2CCN(C3=CC=C(F)C=C3)CC2)CCCCC1,0
...,...,...,...,...,...
6743,c1ccc(CN(CC2=NCCN2)c2ccccc2)cc1,0,False,C1=CC=C(CN(CC2=NCCN2)C2=CC=CC=C2)C=C1,0
6744,CCOCCn1c(N2CCCN(C)CC2)nc2ccccc21,1,False,CCOCCN1C(N2CCCN(C)CC2)=NC2=CC=CC=C21,0
6745,CN1CCC(=C2c3ccccc3CC(=O)c3sccc32)CC1,1,False,CN1CCC(=C2C3=CC=CC=C3CC(=O)C3=C2C=CS3)CC1,0
6746,Cc1[nH]c(=O)c(C#N)cc1-c1ccncc1,0,False,CC1=C(C2=CC=NC=C2)C=C(C#N)C(=O)N1,0


In [51]:
df = df[['SMILES_uncharge', 'Activity']]

In [52]:
df = df.rename(columns={'SMILES_uncharge':'SMILES'})
df.head()

,SMILES,Activity
0,BrC(Br)Br,1
1,C#CC(C)(O)CC,1
2,C#CC(O)(/C=C/Cl)CC,1
3,C#CC(OC(N)=O)C1=CC=CC=C1,1
4,C#CC1(OCC(O)CN2CCN(C3=CC=C(F)C=C3)CC2)CCCCC1,1


### QED and Crippen modules

In [53]:
for i, row in df.iterrows():
    mol = Chem.MolFromSmiles(row.SMILES)
    qed_vector = QED.properties(mol)
    df.at[i, 'MW'] = round(qed_vector[0], 2)
    df.at[i, '#HBA'] = qed_vector[2]
    df.at[i, '#HBD'] = qed_vector[3]
    df.at[i, 'PSA'] = qed_vector[4]
    df.at[i, '#ROTB'] = qed_vector[5]
    df.at[i, '#ALERTS'] = qed_vector[7]

    df.at[i, 'MlogP'] = round(Crippen.MolLogP(mol), 2)
    df.at[i, '#MR'] = round(Crippen.MolMR(mol), 2)

df

,SMILES,Activity,MW,#HBA,#HBD,PSA,#ROTB,#ALERTS,MlogP,#MR
0,BrC(Br)Br,1,252.73,0.0,0.0,0.00,0.0,1.0,2.45,30.57
1,C#CC(C)(O)CC,1,98.14,1.0,1.0,20.23,1.0,1.0,0.78,29.75
2,C#CC(O)(/C=C/Cl)CC,1,144.60,1.0,1.0,20.23,2.0,1.0,1.51,39.07
3,C#CC(OC(N)=O)C1=CC=CC=C1,1,175.19,2.0,1.0,52.32,2.0,1.0,1.46,48.68
4,C#CC1(OCC(O)CN2CCN(C3=CC=C(F)C=C3)CC2)CCCCC1,1,360.47,4.0,1.0,35.94,6.0,1.0,2.66,101.66
...,...,...,...,...,...,...,...,...,...,...
6743,C1=CC=C(CN(CC2=NCCN2)C2=CC=CC=C2)C=C1,0,265.36,2.0,1.0,27.63,5.0,0.0,2.69,84.24
6744,CCOCCN1C(N2CCCN(C)CC2)=NC2=CC=CC=C21,1,302.42,4.0,0.0,33.53,5.0,1.0,2.21,90.55
6745,CN1CCC(=C2C3=CC=CC=C3CC(=O)C3=C2C=CS3)CC1,1,309.43,2.0,0.0,20.31,0.0,0.0,4.01,91.55
6746,CC1=C(C2=CC=NC=C2)C=C(C#N)C(=O)N1,0,211.22,3.0,1.0,69.54,1.0,0.0,1.62,59.75


In [54]:
df

,SMILES,Activity,MW,#HBA,#HBD,PSA,#ROTB,#ALERTS,MlogP,#MR
0,BrC(Br)Br,1,252.73,0.0,0.0,0.00,0.0,1.0,2.45,30.57
1,C#CC(C)(O)CC,1,98.14,1.0,1.0,20.23,1.0,1.0,0.78,29.75
2,C#CC(O)(/C=C/Cl)CC,1,144.60,1.0,1.0,20.23,2.0,1.0,1.51,39.07
3,C#CC(OC(N)=O)C1=CC=CC=C1,1,175.19,2.0,1.0,52.32,2.0,1.0,1.46,48.68
4,C#CC1(OCC(O)CN2CCN(C3=CC=C(F)C=C3)CC2)CCCCC1,1,360.47,4.0,1.0,35.94,6.0,1.0,2.66,101.66
...,...,...,...,...,...,...,...,...,...,...
6743,C1=CC=C(CN(CC2=NCCN2)C2=CC=CC=C2)C=C1,0,265.36,2.0,1.0,27.63,5.0,0.0,2.69,84.24
6744,CCOCCN1C(N2CCCN(C)CC2)=NC2=CC=CC=C21,1,302.42,4.0,0.0,33.53,5.0,1.0,2.21,90.55
6745,CN1CCC(=C2C3=CC=CC=C3CC(=O)C3=C2C=CS3)CC1,1,309.43,2.0,0.0,20.31,0.0,0.0,4.01,91.55
6746,CC1=C(C2=CC=NC=C2)C=C(C#N)C(=O)N1,0,211.22,3.0,1.0,69.54,1.0,0.0,1.62,59.75


### Lipinski module

In [55]:
for i, row in df.iterrows():
    mol = Chem.MolFromSmiles(row.SMILES)
    df.at[i, '#HeavyAtoms'] = Lipinski.HeavyAtomCount(mol)
    df.at[i, '#NHOH'] = Lipinski.NHOHCount(mol)
    df.at[i, '#NO'] = Lipinski.NOCount(mol)
    df.at[i, '#AromaticCarbocycles'] = Lipinski.NumAromaticCarbocycles(mol)
    df.at[i, '#AromaticHeterocycles'] = Lipinski.NumAromaticHeterocycles(mol)
    df.at[i, '#Heteroatoms'] = Lipinski.NumHeteroatoms(mol)

df

,SMILES,Activity,MW,#HBA,#HBD,PSA,#ROTB,#ALERTS,MlogP,#MR,#HeavyAtoms,#NHOH,#NO,#AromaticCarbocycles,#AromaticHeterocycles,#Heteroatoms
0,BrC(Br)Br,1,252.73,0.0,0.0,0.00,0.0,1.0,2.45,30.57,4.0,0.0,0.0,0.0,0.0,3.0
1,C#CC(C)(O)CC,1,98.14,1.0,1.0,20.23,1.0,1.0,0.78,29.75,7.0,1.0,1.0,0.0,0.0,1.0
2,C#CC(O)(/C=C/Cl)CC,1,144.60,1.0,1.0,20.23,2.0,1.0,1.51,39.07,9.0,1.0,1.0,0.0,0.0,2.0
3,C#CC(OC(N)=O)C1=CC=CC=C1,1,175.19,2.0,1.0,52.32,2.0,1.0,1.46,48.68,13.0,2.0,3.0,1.0,0.0,3.0
4,C#CC1(OCC(O)CN2CCN(C3=CC=C(F)C=C3)CC2)CCCCC1,1,360.47,4.0,1.0,35.94,6.0,1.0,2.66,101.66,26.0,1.0,4.0,1.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6743,C1=CC=C(CN(CC2=NCCN2)C2=CC=CC=C2)C=C1,0,265.36,2.0,1.0,27.63,5.0,0.0,2.69,84.24,20.0,1.0,3.0,2.0,0.0,3.0
6744,CCOCCN1C(N2CCCN(C)CC2)=NC2=CC=CC=C21,1,302.42,4.0,0.0,33.53,5.0,1.0,2.21,90.55,22.0,0.0,5.0,1.0,1.0,5.0
6745,CN1CCC(=C2C3=CC=CC=C3CC(=O)C3=C2C=CS3)CC1,1,309.43,2.0,0.0,20.31,0.0,0.0,4.01,91.55,22.0,0.0,2.0,1.0,1.0,3.0
6746,CC1=C(C2=CC=NC=C2)C=C(C#N)C(=O)N1,0,211.22,3.0,1.0,69.54,1.0,0.0,1.62,59.75,16.0,1.0,4.0,0.0,2.0,4.0


### Descriptors module

In [56]:
for i, row in df.iterrows():
    mol = Chem.MolFromSmiles(row.SMILES)
    df.at[i, 'Morgan2'] =  round(Descriptors.FpDensityMorgan2(mol), 2)
    df.at[i, 'Morgan3'] =  round(Descriptors.FpDensityMorgan3(mol), 2)
    df.at[i, 'HeavyAtomMW'] =  round(Descriptors.HeavyAtomMolWt(mol), 2)
    df.at[i, 'MaxPartialCharge'] = Descriptors.MaxPartialCharge(mol)
    df.at[i, 'MinPartialCharge'] = Descriptors.MinPartialCharge(mol)
    df.at[i, '#ValenceElectrons'] = Descriptors.NumValenceElectrons(mol)

df.head()

Выходные данные были обрезаны до нескольких последних строк (5000).
[17:47:48] DEPRECATION WARNING: please use MorganGenerator
[17:47:48] DEPRECATION WARNING: please use MorganGenerator
[17:47:48] DEPRECATION WARNING: please use MorganGenerator
[17:47:48] DEPRECATION WARNING: please use MorganGenerator
[17:47:48] DEPRECATION WARNING: please use MorganGenerator
[17:47:48] DEPRECATION WARNING: please use MorganGenerator
[17:47:48] DEPRECATION WARNING: please use MorganGenerator
[17:47:48] DEPRECATION WARNING: please use MorganGenerator
[17:47:48] DEPRECATION WARNING: please use MorganGenerator
[17:47:48] DEPRECATION WARNING: please use MorganGenerator
[17:47:48] DEPRECATION WARNING: please use MorganGenerator
[17:47:48] DEPRECATION WARNING: please use MorganGenerator
[17:47:48] DEPRECATION WARNING: please use MorganGenerator
[17:47:48] DEPRECATION WARNING: please use MorganGenerator
[17:47:48] DEPRECATION WARNING: please use MorganGenerator
[17:47:48] DEPRECATION WARNING: please use Morg

,SMILES,Activity,MW,#HBA,#HBD,PSA,#ROTB,#ALERTS,MlogP,#MR,...,#NO,#AromaticCarbocycles,#AromaticHeterocycles,#Heteroatoms,Morgan2,Morgan3,HeavyAtomMW,MaxPartialCharge,MinPartialCharge,#ValenceElectrons
0,BrC(Br)Br,1,252.73,0.0,0.0,0.00,0.0,1.0,2.45,30.57,...,0.0,0.0,0.0,3.0,1.00,1.00,251.72,0.124221,-0.063717,26.0
1,C#CC(C)(O)CC,1,98.14,1.0,1.0,20.23,1.0,1.0,0.78,29.75,...,1.0,0.0,0.0,1.0,2.29,2.29,88.06,0.121725,-0.377933,40.0
2,C#CC(O)(/C=C/Cl)CC,1,144.60,1.0,1.0,20.23,2.0,1.0,1.51,39.07,...,1.0,0.0,0.0,2.0,2.44,2.56,135.53,0.144219,-0.373980,50.0
3,C#CC(OC(N)=O)C1=CC=CC=C1,1,175.19,2.0,1.0,52.32,2.0,1.0,1.46,48.68,...,3.0,1.0,0.0,3.0,2.08,2.54,166.11,0.405593,-0.428173,66.0
4,C#CC1(OCC(O)CN2CCN(C3=CC=C(F)C=C3)CC2)CCCCC1,1,360.47,4.0,1.0,35.94,6.0,1.0,2.66,101.66,...,4.0,1.0,0.0,5.0,1.77,2.38,331.24,0.128054,-0.389382,142.0


### GraphDescriptors module

In [57]:
for i, row in df.iterrows():
    mol = Chem.MolFromSmiles(row.SMILES)
    df.at[i, 'BertzCT'] = round(GraphDescriptors.BertzCT(mol), 2)
    df.at[i, 'Kappa1'] = round(GraphDescriptors.Kappa1(mol), 2)
df.head()

,SMILES,Activity,MW,#HBA,#HBD,PSA,#ROTB,#ALERTS,MlogP,#MR,...,#AromaticHeterocycles,#Heteroatoms,Morgan2,Morgan3,HeavyAtomMW,MaxPartialCharge,MinPartialCharge,#ValenceElectrons,BertzCT,Kappa1
0,BrC(Br)Br,1,252.73,0.0,0.0,0.00,0.0,1.0,2.45,30.57,...,0.0,3.0,1.00,1.00,251.72,0.124221,-0.063717,26.0,8.00,5.44
1,C#CC(C)(O)CC,1,98.14,1.0,1.0,20.23,1.0,1.0,0.78,29.75,...,0.0,1.0,2.29,2.29,88.06,0.121725,-0.377933,40.0,86.84,6.52
2,C#CC(O)(/C=C/Cl)CC,1,144.60,1.0,1.0,20.23,2.0,1.0,1.51,39.07,...,0.0,2.0,2.44,2.56,135.53,0.144219,-0.373980,50.0,145.49,8.55
3,C#CC(OC(N)=O)C1=CC=CC=C1,1,175.19,2.0,1.0,52.32,2.0,1.0,1.46,48.68,...,0.0,3.0,2.08,2.54,166.11,0.405593,-0.428173,66.0,326.53,9.14
4,C#CC1(OCC(O)CN2CCN(C3=CC=C(F)C=C3)CC2)CCCCC1,1,360.47,4.0,1.0,35.94,6.0,1.0,2.66,101.66,...,0.0,5.0,1.77,2.38,331.24,0.128054,-0.389382,142.0,599.43,19.16


# **All_descriptors**


In [58]:
from rdkit.ML.Descriptors import MoleculeDescriptors

In [59]:
def get_all_descriptors(smiles):
    mols = [Chem.MolFromSmiles(i) for i in smiles]
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    desc_names = calc.GetDescriptorNames()
    mol_descriptors = []
    for mol in tqdm(mols):
        mol = Chem.AddHs(mol)
        descriptors = calc.CalcDescriptors(mol)
        mol_descriptors.append(descriptors)
    return mol_descriptors, desc_names

In [60]:
print(df['SMILES'].describe())
mol_descriptors, descriptors_names = get_all_descriptors(df['SMILES'].tolist()) #вызов функции для подсчета дескрипторов
df[[*descriptors_names]] = mol_descriptors # добавление дескрипторов в датасет

count                                             6748
unique                                            6745
top       CC1=C(C)C(C)=C(C(C)(C)N(C(C)(C)N)C(C)(C)N)O1
freq                                                 2
Name: SMILES, dtype: object


Выходные данные были обрезаны до нескольких последних строк (5000).
****

[17:59:03] 

****
Pre-condition Violation
bad result vector size
Violation occurred on line 40 in file /project/build/temp.linux-x86_64-cpython-310/rdkit/Code/GraphMol/Descriptors/Crippen.cpp
Failed Expression: logpContribs.size() == mol.getNumAtoms() && mrContribs.size() == mol.getNumAtoms()
----------
Stacktrace:
----------
****

[17:59:03] 

****
Pre-condition Violation
bad result vector size
Violation occurred on line 40 in file /project/build/temp.linux-x86_64-cpython-310/rdkit/Code/GraphMol/Descriptors/Crippen.cpp
Failed Expression: logpContribs.size() == mol.getNumAtoms() && mrContribs.size() == mol.getNumAtoms()
----------
Stacktrace:
----------
****

[17:59:03] 

****
Pre-condition Violation
bad result vector size
Violation occurred on line 40 in file /project/build/temp.linux-x86_64-cpython-310/rdkit/Code/GraphMol/Descriptors/Crippen.cpp
Failed Expression: logpContribs.size() == mol.getNumAtoms() && mrC

In [61]:
from sklearn.feature_selection import mutual_info_classif

In [62]:
def create_feature_importances(X, y):
    mutual_info = mutual_info_classif(X, y)
    threshold = 0.025
    selected_features = X.columns[mutual_info < threshold]
    X_selected = X[selected_features]
    print(f"Изначальное количество признаков: {X.shape[1]}")
    print(f"Количество отобранных признаков: {X_selected.shape[1]}")
    return X_selected
X = df.drop('Activity', axis=1)
X = X.fillna(0).drop('SMILES', axis=1)
y = df['Activity']
X = create_feature_importances(X, y)

Изначальное количество признаков: 228
Количество отобранных признаков: 96


### FINGERPRINTS


In [17]:
from rdkit.Avalon import pyAvalonTools

In [64]:
def generate_AVfpts(data):
    Avalon_fpts = []
    mols = [Chem.MolFromSmiles(x) for x in data if x is not None]
    for mol in tqdm(mols):
        avfpts = pyAvalonTools.GetAvalonFP(mol, nBits=512)
        Avalon_fpts.append(avfpts)
    return pd.DataFrame(np.array(Avalon_fpts))
Avalon_fpts = generate_AVfpts(df['SMILES'])
Avalon_fpts

100%|██████████| 6748/6748 [00:10<00:00, 626.63it/s]


,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,1,0,0,0,0,1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6743,0,0,0,0,1,1,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0
6744,1,0,1,0,1,1,0,0,0,1,...,0,0,1,0,0,1,1,1,1,1
6745,1,0,0,0,1,1,0,0,0,1,...,1,0,1,1,0,1,0,0,1,0
6746,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,1,0,0,1


In [74]:
X_new = pd.concat([X, Avalon_fpts], axis=1, ignore_index=True)
X_new.columns = X_new.columns.astype(str)
X_new

,0,1,2,3,4,5,6,7,8,9,...,598,599,600,601,602,603,604,605,606,607
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6743,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,1,0,0,1,0
6744,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,1,1,1,1,1
6745,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,1,1,0,1,0,0,1,0
6746,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,1,0,0,1


In [71]:
X_new1 = df[["SMILES", "Activity"]]
X_new2 = pd.concat([X_new1, X_new], axis=1)

In [72]:
X_new2.to_csv('fragments_classification_actual_wth_modules.csv', index=False)